In [ ]:
# | default_exp routes.role

In [ ]:
# | export
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

In [1]:
#| hide
import os
import pandas as pd
import datetime as dt


# Get and Search Routes


In [ ]:
# | export
class Role_NotRetrieved(de.DomoError):
    def __init__(
        self,
        domo_instance,
        function_name,
        status,
        message,
        role_id=None,
        parent_class=None,
    ):
        super().__init__(
            domo_instance=domo_instance,
            entity_id=role_id,
            function_name=function_name,
            status=status,
            message=message,
            parent_class=parent_class,
        )


# | export
class Role_CRUD_Error(de.DomoError):
    def __init__(
        self,
        domo_instance,
        function_name,
        status,
        message,
        role_id=None,
        parent_class=None,
    ):
        super().__init__(
            domo_instance=domo_instance,
            entity_id=role_id,
            function_name=function_name,
            status=status,
            message=message,
            parent_class=parent_class,
        )

In [ ]:
# | export
@gd.route_function
async def get_roles(
    auth: dmda.DomoAuth,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/authorization/v1/roles"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        session=session,
    )

    if not res.is_success:
        raise Role_NotRetrieved(
            domo_instance=auth.domo_instance,
            function_name=res.traceback_details.function_name,
            status=res.status,
            message=res.response,
            parent_class=parent_class,
        )

    return res

#### sample implementation of get_roles


In [ ]:
#| eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_roles(auth=token_auth)

pd.DataFrame(res.response[0:5])

,id,name,description,authorityCount,userCount,created,modified
0,1,Admin,Full access to everything,83,19,1580425515000,1580425515000
1,2,Privileged,Full access except for editing users and setti...,34,628,1580425515000,1580425515000
2,3,Editor,"Can edit Cards, Pages, DataSets, and Dataflows",30,1,1580425515000,1580425515000
3,4,Participant,Read only access to Cards and Pages,6,4,1580425515000,1580425515000
4,5,Social,Access limited to social features,1,16,1580425515000,1580425515000


In [ ]:
# | export


@gd.route_function
async def get_role_by_id(
    auth: dmda.DomoAuth,
    role_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class: str = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        session=session,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if not res.is_success:
        raise Role_NotRetrieved(
            domo_instance=auth.domo_instance,
            function_name=res.traceback_details.function_name,
            status=res.status,
            message=res.response,
            parent_class=parent_class,
        )

    return res

In [ ]:
#| eval : false 

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await get_role_by_id(auth=token_auth, role_id=1)

ResponseGetData(status=200, response={'id': 1, 'name': 'Admin', 'description': 'Full access to everything', 'authorityCount': 83, 'userCount': 19, 'created': 1580425515000, 'modified': 1580425515000}, is_success=True, parent_class=None)

In [ ]:
# | export


@gd.route_function
async def get_role_grants(
    auth: dmda.DomoAuth,
    role_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class: str = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}/authorities"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        session=session,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if len(res.response) == 0:
        role_res = await get_roles(auth=auth)

        domo_role = [role for role in role_res.response if role.get("id") == role_id]

        if not domo_role:
            raise Role_NotRetrieved(
                domo_instance=auth.domo_instance,
                function_name=res.traceback_details.function_name,
                message=f"role {role_id} does not exist",
                status=res.status,
                parent_class=parent_class,
            )

    return res

#### sample implementation of get_role_grants


In [ ]:
#| eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_role_grants(auth=token_auth, role_id=1)

pd.DataFrame(res.response[0:5])

,0
0,versions.deployment.manage
1,ai.model.create
2,buzz.admin
3,dataflow.admin
4,goal.admin


In [ ]:
# | export


@gd.route_function
async def get_role_membership(
    auth: dmda.DomoAuth,
    role_id: str,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}/users"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        session=session,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if len(res.response.get("users")) == 0:
        role_res = await get_roles(auth)

        domo_role = next(
            (role for role in role_res.response if role.get("id") == role_id), None
        )

        if not domo_role:
            raise Role_NotRetrieved(
                domo_instance=auth.domo_instance,
                function_name=res.traceback_details.function_name,
                message=f"role {role_id} does not exist or cannot be retrieved",
                status=res.status,
                parent_class=parent_class,
            )

    if return_raw:
        return res

    res.response = res.response.get("users")

    return res

#### sample implementation of get_role_membership


In [ ]:
#| eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_role_membership(auth=token_auth, role_id=1, return_raw=False)

pd.DataFrame(res.response[0:5])

,userId,email,displayName,roleId,department,jobTitle,hireDate,location
0,1345408759,DataMaven42@gmail.com,Alexis Lorenz (DataMaven),1,Owner,Major Domo Support Specialist,1.590156e+12,NaN
1,1102391116,andrea.henderson@domo.com,Andrea Henderson,1,NaN,NaN,NaN,NaN
2,1095515127,andrew@crystalballers.ai,Andrew Chaffin,1,Crystal Ballers,Cofounder,NaN,
3,1865099622,Ben.Schein@domo.com,Ben Schein,1,NaN,NaN,NaN,NaN
4,587894148,bryan@bvankampen.net,Bryan Van Kampen,1,BT Partners,Executive Analytics,1.588828e+12,NaN


# CRUD Routes


In [ ]:
# | export


@gd.route_function
async def create_role(
    auth: dmda.DomoAuth,
    name: str,
    description: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/authorization/v1/roles"

    body = {"name": name, "description": description}

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="POST",
        body=body,
        session=session,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if not res.is_success:
        raise Role_CRUD_Error(
            domo_instance=auth.domo_auth,
            function_name=res.traceback_deails.function_name,
            status=res.status,
            message=res.response,
        )

    return res

In [ ]:
# | export


@gd.route_function
async def delete_role(
    auth: dmda.DomoAuth,
    role_id: int,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="DELETE",
        session=session,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if not res.is_success:
        raise Role_CRUD_Error(
            domo_instance=auth.domo_auth,
            function_name=res.traceback_deails.function_name,
            status=res.status,
            message=res.response,
        )

    res.response = f"role {role_id} deleted or doesn't exist"

    return res

#### sample implementation of delete_role


In [ ]:
#| eval : false

# import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await delete_role(auth=token_auth, role_id="2014090557")

ResponseGetData(status=204, response="role 2014090557 deleted or doesn't exist", is_success=True, parent_class=None)

### Default Role


In [ ]:
# | export


@gd.route_function
async def get_default_role(
    auth,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
):
    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/customer-states/user.roleid.default"

    params = {"defaultValue": 2, "ignoreCache": True}

    res = await gd.get_data(
        auth=auth,
        method="GET",
        url=url,
        params=params,
        session=session,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if not res.is_success:
        raise Role_NotRetrieved(
            domo_instance=auth.domo_auth,
            function_name=res.traceback_deails.function_name,
            status=res.status,
            message=res.response,
        )

    res.response = res.response.get("value")

    return res

#### sample implementation of get_default_role


In [ ]:
#| eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await get_default_role(token_auth)

ResponseGetData(status=200, response='2', is_success=True, parent_class=None)

In [ ]:
# | export


@gd.route_function
async def set_default_role(
    auth: dmda.DomoAuth,
    role_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class=None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/customer-states/user.roleid.default"

    body = {"name": "user.roleid.default", "value": role_id}

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        debug_api=debug_api,
        body=body,
        session=session,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if not res.is_success:
        raise Role_CRUD_Error(
            domo_instance=auth.domo_auth,
            function_name=res.traceback_deails.function_name,
            status=res.status,
            message=res.response,
        )

    return res

### Metadata


In [ ]:
# | export
@gd.route_function
async def update_role_metadata(
    auth: dmda.DomoAuth,
    role_id,
    role_name,
    role_description: str = None,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
):
    url = f"https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}"

    body = {"name": role_name, "description": role_description, "id": role_id}

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        session=session,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if not res.is_success:
        raise Role_CRUD_Error(
            domo_instance=auth.domo_auth,
            function_name=res.traceback_deails.function_name,
            status=res.status,
            message=res.response,
        )

    return res

#### sample update_role_metadata


In [ ]:
#| eval : false

# import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

# get role to update
role_ls = await get_roles(auth=token_auth)
role = next(
    (role for role in role_ls.response if role.get("name") == "super_admin"), None
)


# update role
await update_role_metadata(
    auth=token_auth,
    role_id=role["id"],
    role_name=role["name"],
    role_description=f"last updated - {dt.date.today()}",
)

# validate role has been updated
role_ls = await get_roles(auth=token_auth)
role = next(
    (role for role in role_ls.response if role.get("name") == "super_admin"), None
)

role

{'id': 810756122,
 'name': 'super_admin',
 'description': 'last updated - 2023-12-08',
 'authorityCount': 90,
 'userCount': 3,
 'created': 1678842530000,
 'modified': 1702046774000}

In [ ]:
# | export


@gd.route_function
async def set_role_grants(
    auth: dmda.DomoAuth,
    role_id: str,
    role_grant_ls: list[str],
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}/authorities"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=role_grant_ls,
        session=session,
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if not res.is_success:
        raise Role_CRUD_Error(
            domo_instance=auth.domo_auth,
            function_name=res.traceback_deails.function_name,
            status=res.status,
            message=res.response,
        )

    return res

In [ ]:
# | export


@gd.route_function
async def role_membership_add_users(
    auth: dmda.DomoAuth,
    role_id: str,
    user_list: list[str],  # list of user ids
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}/users"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=user_list,
        session=session,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if not res.is_success:
        raise Role_CRUD_Error(
            domo_instance=auth.domo_auth,
            function_name=res.traceback_deails.function_name,
            status=res.status,
            message=res.response,
        )

    return res

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()